Note: This assumes that the `mantidimaging` is available on the PATH, otherwise the subprocess commands will not be able to execute the package successfully. On how to do that please set up global access to the package https://mantidproject.github.io/mantidimaging/user_guide/setting_up.html

In [ ]:
# Setup the package, this is done automatically with the startup scripts
import os
import sys

# Setup some constants
HOME = os.path.expanduser('~')
CHADWICK_PATH = os.path.join(HOME, 'mantidimaging', 'notebooks', 'demoimages')
CHADWICK_SAMPLE_PATH = os.path.join(CHADWICK_PATH, 'sample')
SINOGRAMS_PATH = os.path.join(CHADWICK_PATH, 'sinograms')
SINOGRAMS_MLOG_PATH = os.path.join(CHADWICK_PATH, 'sinograms_mlog')

CHADWICK_PREPROC_PATH = os.path.join(CHADWICK_PATH, 'preproc')
CHADWICK_PREPROC_MLOG_PATH = os.path.join(CHADWICK_PATH, 'preproc-mlog')

MANUAL_COR_OUTPUT_PATH = os.path.join(CHADWICK_PATH, 'cors')

RECONSTRUCTION_OUTPUT_PATH = os.path.join(CHADWICK_PATH, 'reconstructed')
RECONSTRUCTION_MLOG_OUTPUT_PATH = os.path.join(CHADWICK_PATH, 'reconstructed_mlog')

MANTIDIMAGING_PATH = os.path.join(HOME, 'mantidimaging')
print(HOME, CHADWICK_PATH, MANTIDIMAGING_PATH)
sys.path[0]=MANTIDIMAGING_PATH

import mantidimaging
print(mantidimaging.__package__)

# Setup matplotlib to use the notebook backend
import matplotlib
matplotlib.use('nbagg')
from matplotlib import pyplot

# Create matplotlib objects
def show(image, idx=0):
    fig, img_axes = pyplot.subplots(nrows=1,ncols=1)
    
    image_obj = img_axes.imshow(image.get_sample()[idx], cmap='Greys_r')

    pyplot.show()

# import package from isis_imaging's GUI part
from mantidimaging.gui.stack_visualiser import sv_histogram

# python abuse to plot the histograms on the same plot, it works by default in the package

def plot_histograms_magically(data, idx1=0, idx2=None):
    # save the original function reference
    if idx2:
        temp_func_storage = sv_histogram._show
        def _temporary_empty_function(): pass
        # set to a function that does nothing
        sv_histogram.show_transparent(data.get_sample()[idx2].flatten(), "Normal Contrast", "")

        # restore so that the plot is shown next time
        sv_histogram._show = temp_func_storage

    sv_histogram.show_transparent(data.get_sample()[idx1].flatten(), "Low Contrast", "Magic")    

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>
# Load some data in

In [ ]:
# Load some data in, format for indices is [start, end, step]
indices = [0, 2, 1]

# use the package directly in a script/ipython/notebook style
images = mantidimaging.core.io.loader.load(CHADWICK_PREPROC_PATH, indices=indices)
show(images)

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>
# Finding the COR

In [ ]:
# only run on 2 images
import subprocess

# Another interface to the package: the CLI, used on SCARF
# does a command line call, exactly as if I typed it in the terminal. 

print("Command line executed:\n", 
      " ".join(['mantidimaging', '-i', CHADWICK_PREPROC_PATH, '--indices', '0', '2', '1', '--imopr', 'cor']),
     "\n")

# This is the interface that will be used in the Reconstruction part
result = subprocess.run(['mantidimaging', '-i', CHADWICK_PREPROC_PATH, \
                         '--indices', '0', '2', '1', \
                         '--imopr', 'cor'], stdout=subprocess.PIPE)

if result.stdout:
    print(result.stdout.decode())
if result.stderr:
    print(result.stderr.decode())

The calculated center of rotation can be found with the prefix 

```
Running COR for index * [ ...]`
```

A thing to note here is the message 

```text
This works ONLY with sinograms
```

because what we're running the algorithm with is the _projections_. There is no way to automatically determine if we have a projection or a sinogram, as the dimensons vary depending on crop.

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>

# Converting to Sinograms

<br>
## Sinogram - a row from each image, shows the movement of the row throughout the images
<br>

### This means that for sinograms you need _ALL_ of the information along the Z axis

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>
# Visualise a projection

In [ ]:
# show one of the first projections
show(images) 

# How does a sinogram look?

In [ ]:
sinogram_indices = [0, 2, 1]
sinograms = mantidimaging.core.io.loader.load(SINOGRAMS_PATH, indices=sinogram_indices)

# shows the 600th sinogram. This is the 600th row of EVERY projection 
# from the image volume and shows the movement of the object through the volume
show(sinograms) 

The following would be the code to convert the projections to sinograms, however that requires loading the whole dataset, but not all images are provided for the demo.

```python

import subprocess
result = subprocess.run(['mantidimaging', '-i', CHADWICK_SAMPLE_PATH, \
                        '-o', SINOGRAMS_PATH, '--convert', '--swap-axes'], stdout=subprocess.PIPE)
print(result.stdout.decode())
```

In [ ]:
print("The command line to convert the projections to sinograms is:\n", " ".join(['mantidimaging', '-i', CHADWICK_SAMPLE_PATH, '-o', SINOGRAMS_PATH, '--convert', '--swap-axes']))

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>
# Why do we work with Sinograms and not the original slices?
Tomopy will implicitly convert the data to sinograms, if we send projections. This doubles the memory usage (and loses time converting the data). If we pass sinograms, we don't have surprise memory doubling.

It is specified through a `sinogram_order=True` flag in the actual Tomopy call:
```python
    recon = self._tomopy.recon(
        tomo=sample,
        theta=proj_angles,
        center=cors,
        ncore=cores,
        algorithm=alg,
        sinogram_order=True,
        **kwargs)
```

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>
# How to use automatic approximation of COR

In [ ]:
print("Command line:\n", " ".join(['mantidimaging', '-i', SINOGRAMS_PATH, \
                         '--indices', '550', '600', '500', \
                         '--imopr', 'cor']))

import subprocess
result = subprocess.run(['mantidimaging', '-i', SINOGRAMS_PATH, \
                         '--indices', '0', '2', '1', \
                         '--imopr', 'cor'], stdout=subprocess.PIPE)
if result.stdout:
    print(result.stdout.decode())
if result.stderr:
    print(result.stderr.decode())

The center of rotation is a lot closer to the actual one when run on the sinogram (the one I found and which we will use below is 134).

The algorithm is generally fairly innacurate for the high noise datasets from neutron tomography.

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>
# How to use manual finding of COR

The centers of rotations we are reconstructing are specified after the `--imopr` flag.

We will reconstruct the CORs starting 130 to 140, with a step of 1. The output will be saved out in the specified directory with `-o`.

In [ ]:
import subprocess
result = subprocess.run(['mantidimaging', '-i', SINOGRAMS_PATH, \
                         '-o', MANUAL_COR_OUTPUT_PATH, \
                         '--indices', '0', '2', '1', \
                         '--imopr', '130', '140', '1', 'corwrite'], stdout=subprocess.PIPE)

if result.stdout:
    print(result.stdout.decode())
if result.stderr:
    print(result.stderr.decode())

In [ ]:
recon_slices = mantidimaging.core.io.loader.load(os.path.join(MANUAL_COR_OUTPUT_PATH, '0'), in_format='tiff')

# There is not much difference between going below or over the COR
show(recon_slices, 0) # too low COR

In [ ]:
# too high COR
show(recon_slices, 9)

In [ ]:
# Closer to the correct COR, but still a little bit of shadow left
show(recon_slices, 6)

In [ ]:
# just right
show(recon_slices, 4)

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>
# Running a reconstruction
## Setting up the reconstruction parameters

In [ ]:
# Correct Reconstruction COR parameters
SLICE_CORS = [
  # Slice ID, Center of Rotation
    ('422',       '542'), 
    ('822',       '540'), 
    ('1222',      '540'), 
    ('1622',      '537'), 
    ('1822',      '536')
]

# Python 3 needs the list(...) as a gentle nudge, because zip is a generator
SLICE_IDS, CORS = list(zip(*SLICE_CORS))
print("Slices", SLICE_IDS)
print("Centers of Rotation", CORS)

## Trying to run a reconstruction

In [ ]:
# Keep the same Centers of Rotation
# But add one more to the slice indices
# Now we have a slice that doesn't have a COR associated with it
WRONG_SLICE_IDS = (*SLICE_IDS, '1922',)
print(WRONG_SLICE_IDS)


# This will crash because we have not provided the same amount of CORs and slice IDs

import subprocess
result = subprocess.run(['mantidimaging', '-i', SINOGRAMS_PATH, \
                         '-o', RECONSTRUCTION_OUTPUT_PATH, \
                         '--reconstruction', \
                         '--cors', *CORS, \
                         '--cor-slices', *WRONG_SLICE_IDS, \
                         '-t', 'tomopy', \
                         '-a', 'gridrec'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print(result.stderr.decode())

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>
## Actually running the reconstruction
## Note: For visual comparison this data DOES NOT have Minus Log applied to it!

In [ ]:
SLICE_CORS = [
  # Slice ID, Center of Rotation
    ('0',       '134'), 
    ('1',       '134')
]

SLICE_IDS, CORS = list(zip(*SLICE_CORS))
import subprocess
result = subprocess.run(['mantidimaging', '-i', SINOGRAMS_PATH, \
                         '-o', RECONSTRUCTION_OUTPUT_PATH, \
                         '--reconstruction', \
                         '--cors', *CORS, \
                         '--cor-slices', *SLICE_IDS, '-w'], stdout=subprocess.PIPE)
if result.stdout:
    print(result.stdout.decode())
if result.stderr:
    print(result.stderr.decode())

In [ ]:
path = os.path.join(RECONSTRUCTION_OUTPUT_PATH, 'reconstructed')

root, dirs, files = next(os.walk(path))
print("\n".join(sorted(files)))

In [ ]:
recon = mantidimaging.core.io.loader.load(path, indices=[0, 2, 1])

show(recon)

# Minus Log reconstruction

In [ ]:
SLICE_CORS = [
  # Slice ID, Center of Rotation
    ('0',       '134'), 
    ('1',       '134')
]

SLICE_IDS, CORS = list(zip(*SLICE_CORS))
import subprocess
result = subprocess.run(['mantidimaging', '-i', SINOGRAMS_MLOG_PATH, \
                         '-o', RECONSTRUCTION_MLOG_OUTPUT_PATH, \
                         '--reconstruction', \
                         '--cors', *CORS, \
                         '--cor-slices', *SLICE_IDS, '-w'], stdout=subprocess.PIPE)
if result.stdout:
    print(result.stdout.decode())
if result.stderr:
    print(result.stderr.decode())

In [ ]:
mlog_path = os.path.join(RECONSTRUCTION_MLOG_OUTPUT_PATH, 'reconstructed')
recon = mantidimaging.core.io.loader.load(mlog_path, indices=[0, 2, 1])

show(recon)